In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py

import numpy as np
import scipy

# Triangular solver

Based on scipy's [scipy.linalg.solve_triangular](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_triangular.html)

Having a lower triangular matrix results in a _forward_ substitution, while an upper triangular matrix results in a _backward_ substitution.

_Note:_ this was developed expecting uncompressed tensors but its use of `getPayload{,Ref}()` should support compressed formats.

In [ ]:
M = N = 8

np.random.seed(12345)

# Generate a random upper triangular matrix A
A_data = np.tril(np.random.randint(1, 6, size=(M, N)))

# Generate the desired output vector x (Ax=b)
X_ref_data = np.array(np.random.randint(1, 6, size=(N,)))

# Evaluate the triangular matrix with X_ref to obtain B
B_data = A_data.dot(X_ref_data)

print("A:\n", A_data, "\nx (reference):\n", X_ref_data, "\nb:\n", B_data)

assert(all(X_ref_data == scipy.linalg.solve_triangular(A_data, B_data, lower=True)))

In [ ]:
# Create uncompressed tensors
A_MN = Tensor.fromUncompressed(["M", "N"], A_data.tolist()).setName("A")
B_N = Tensor.fromUncompressed(["N"], B_data.tolist()).setName("b")
X_ref = Tensor.fromUncompressed(["M"], X_ref_data.tolist()).setName("x_ref")
X = Tensor(rank_ids=["M"]).setName("x")

A = A_MN.getRoot()

inflate = False
if inflate:
    # hack to "inflate" A with coordinates where there are zeros,
    # and allowing us to use square brackets (i.e. __getitem__)
    # because positions == coordinates
    A << Fiber(coords=range(M))
    for j, a_j in A:
        a_j << Fiber(coords=range(N))

B = B_N.getRoot()

displayTensor(A)
displayTensor(B)

## Pipelined triangular solver

`P` is a representation of pipelined communication; each row corresponds to the data passed from the previous stage to the next. Each row is N elements wide (same as B).

This mapping corresponds to a pipeline of single-threaded compute engines (CEs) commensurate in size to the input.

_Note:_ I traverse A in the N rank, then the M rank. I think I need a `swapRanks()` somewhere...

In [ ]:
# create "pipes" between each stage: N-entry tensors
# (can't simply do a rank-2 tensor because I'd have multiple entries for addActivity for that tensor, and
# current pipelining API doesn't really handle that
#
# for syntactic cleanliness I include B, the input tensor, in this array too
BP = [B_N] + [Tensor(rank_ids=["N"], shape=[N]).setName(f"pipe-{m}") for m in range(M)]

canvas = createCanvas(A_MN, *BP, X, enable_wait=True)

cycle = 0
stage_delay = 1

# "wait" means, for each key-value pair in the supplied dictionary,
# we must wait `value` cycles when the `key`th argument in the canvas has been updated 
# in order to add our own activity
wait = None # loop-carried variable

for a_m in range(M):
    # in the first row, read from B, rather than P
    # also, skew these starts by cycle, as these input elements are piped in one after another
    # m_ and n_activities provide pair-wise activity locations for adjacent stages in the pipeline
    # e.g., [(m,), (m,), ()] then [(), (m,), (m,)] then [(), (), (m,)]
    m_activities = [(a_m,) if mm == a_m or mm == a_m+1 else () for mm in range(M+1)]
    # print(m_activities)

    # process element on diagonal
    curStage = BP[a_m]
    nextStage = BP[a_m+1]
    a_n = a_m
    next_m = nextStage.getPayloadRef(a_m)
    next_m <<= curStage.getPayload(a_m) / A.getPayload(a_m, a_n)
    addActivity(canvas, (a_n, a_m), *m_activities, worker=str(a_m), skew=(cycle if a_m == 0 else 0), wait=wait)
    cycle += 1
    
    # process elements off-diagonal on same row
    # change from a_m to a_n (even though M=N) for clarity
    for i_n in range(a_n+1, N):
        n_activities = [(i_n,) if ii == a_n or ii == a_n+1 else () for ii in range(N+1)]
        # print(n_activities)
        
        cur_n = curStage.getPayload(i_n)
        next_n = nextStage.getPayloadRef(i_n)
        next_n <<= cur_n - next_m * A.getPayload(i_n, a_m)

        addActivity(canvas, (i_n, a_m), *n_activities, worker=str(a_m), skew=(cycle if a_m == 0 else 0), wait=wait)
        cycle += 1
        
    # new values for next value of a_m
    wait = {f"pipe-{a_m}": stage_delay}

for x_m in range(M):
    x_m_ref = X.getPayloadRef(x_m)
    x_m_ref <<= BP[x_m+1].getPayload(x_m) # get m-th value from pipeline stage m
    m_activities = [(x_m,) if mm == x_m else () for mm in range(M)]
    # print(x_m, [], [], m_activities + [(x_m,)])
    addActivity(canvas, [], [], *m_activities, (x_m,), wait={f"pipe-{x_m}":stage_delay})
            
displayCanvas(canvas)


In [ ]:
# final result
displayTensor(X)
displayTensor(X_ref)
X == X_ref

## Check results

Check the result by performing the dot product of `A` and `x` and ensuring that equals the `b` we started with. (The jth entry of `b` corresponds to the dot product of the jth _column_ of `A` and `x`.)

In [ ]:
B_check = Fiber(coords=range(N), initial=0)
# displayTensor(B2)

# canvas2 = createCanvas(A, X, B2)
for j in range(N):
    for i in range(N):
        # print(j, i, a_pay[i], x_val, b2_ref)
        b2_ref = B_check.getPayloadRef(j)
        x_val = X.getPayload(i)
        b2_ref += A.getPayload(j,i) * x_val 
        # addActivity(canvas2, (i,j), (i,), (i,))
        
# displayCanvas(canvas2)

# check results
B == B_check

# Tiling schemes

For your convenience, `A_MN` and `B_N` are repeated here:

In [ ]:
displayTensor(A_MN)
displayTensor(B_N)

### A_MMNN

In [ ]:
A_MMNN = A_MN.splitUniform(2).splitUniform(2, depth=2)
displayTensor(A_MMNN)

### A_MNMN

In [ ]:
A_MNMN = A_MMNN.swapRanks(depth=1)
displayTensor(A_MNMN)

### B_NN

In [ ]:
B_NN = B_N.splitUniform(2)
displayTensor(B_NN)